# CIA Factbook Data Analysis with SQL

The CIA factbook, also known as the [World Factbook](https://www.cia.gov/the-world-factbook/), is an annual publication of the US Central Intelligence Agency. It provides basic intelligence by summarizing information about countries and regions worldwide. The factbook contains a mix of demographic and geographic data, among many more.

This project aims to use SQL in Jupyter Notebook to analyze data from this [SQLite factbook.db](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db) database.

## Connecting to the Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Understanding the Data
---
First, let's examine the tables in our database:

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


#### Notes
>- There are two tables in the database, **sqlite_sequence** and **facts**. 
>- The sqlite_sequence table does not contain any columns. 
>- The facts table contains information on _population, birth rate, migration rate_ and many more: this is what we need.
  
We will work with the **facts** table henceforth.

In [3]:
%%sql
-- Examine the first five rows in facts table
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The facts table comprises **11 columns** with intuitive names:
>- **id** - Entry row number.
>- **code** — The country's [internet code](https://www.cia.gov/the-world-factbook/field/internet-country-code/).
>- **name** — Name of the country.
>- **area** — The country's total area (both land and water).
>- **area_land** — The country's land area in square kilometers.
>- **area_water** — The country's water area in square kilometers.
>- **population** —  Whole number of people or inhabitants in the country.
>- **population_growth** — The country's population growth as a percentage.
>- **birth_rate** — The number of births per year per 1,000 inhabitants.
>- **death_rate** — The number of deaths per year per 1,000 inhabitants.
>- **migration_rate** — The difference between the number of persons entering and leaving the country during the year per 1,000 persons. 
  
Finally, let's check the total number of records in the facts table:

In [4]:
%%sql
-- Calculate the total number of records
SELECT COUNT(*) AS num_records,
       COUNT(DISTINCT name) AS unique_countries
  FROM facts;

 * sqlite:///factbook.db
Done.


num_records,unique_countries
261,261


261 records and countries seem unintuitive! We know there are only 195 countries in the world right now. 

The answer lies in the way the CIA categorizes its information. As of July 2011, The World Factbook comprises 267 **entities** divided into the following categories:
>- **Independent countries:** The CIA defines these as people "politically organized into a sovereign state with a definite territory." There are 195 entities in this category.
>- **Others:** Places set apart from the list of independent countries. Currently, there are two: Taiwan and the European Union.
>- **Dependencies and Areas of Special Sovereignty:** Places affiliated with another country. There are 58 in total.
>- **Miscellaneous:** Antarctica and places in dispute. There are six such entities.
>- **Other entities:** The world and the oceans. There are five oceans and the world (the World entry is intended as a summary of the other entries).

Additional information about this classification can be found in the [world factbook wikipedia page](https://en.wikipedia.org/wiki/The_World_Factbook#:~:text=Entities%20listed,-Main%20article%3A%20List&text=The%20CIA%20defines%20these%20as,category%2C%20there%20are%20195%20entities.&text=Places%20set%20apart%20from%20the,Taiwan%20and%20the%20European%20Union.).

We can check for these occurrences in our data:

In [5]:
%%sql
-- check for some instances of "non-country" data
SELECT *
  FROM facts
 WHERE name LIKE '%union' 
    OR name LIKE '%ocean'
    OR name LIKE '%island'
    OR name = 'World';

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
199,kt,Christmas Island,135,135,0,1530,1.11,None,None,None
203,nf,Norfolk Island,36,36,0,2210,0.01,None,None,None
208,ip,Clipperton Island,6,6,0,None,None,None,None,None
222,bv,Bouvet Island,49,49,0,None,None,None,None,None
244,bq,Navassa Island,5,5,0,None,None,None,None,None
248,wq,Wake Island,6,6,0,None,None,None,None,None
256,xq,Arctic Ocean,None,None,None,None,None,None,None,None
257,zh,Atlantic Ocean,None,None,None,None,None,None,None,None
258,xo,Indian Ocean,None,None,None,None,None,None,None,None


>- Since these occurrences are present, we should be careful to avoid their influence on the analysis process.
  
We can now proceed to analysis with _some_ understanding of the data and potential pitfalls to look out for.

## Exploring Total Population
---
To understand how population values are distributed in the data, we can compute the population summary statics: the minimum, mean and maximum.

In [6]:
%%sql 
SELECT MIN(population) AS min_population,
       CAST(AVG(population) AS Integer) AS avg_population,
       MAX(population) AS max_pop
  FROM facts;

 * sqlite:///factbook.db
Done.


min_population,avg_population,max_pop
0,62094928,7256490011


Surprising! Some countries/entities have no inhabitants. Another entity has a population of over 7 billion people. This population seems too good to be true since the world population is currently at [7.96 billion](https://www.worldometers.info/world-population/#:~:text=World%20Population%20Clock%3A%207.96%20Billion%20People%20(2022)%20%2D%20Worldometer) _(Worldometer Aug, 2022)_. It is best to identify what these entities are:

In [7]:
%%sql 
-- Investigate the unusual countries/entities
SELECT name, population
  FROM facts
 WHERE population = (SELECT MIN(population) FROM facts) 
    OR population = (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0
World,7256490011


Similar to earlier observations, the presence of entries like _World_ and _Antarctica_ explains the unusual population values. According to the [World population review](https://worldpopulationreview.com/continents/antarctica-population), Antarctica has no indigenous inhabitants, only research staff occasionally present at its many research stations. 

Another interesting thing to note is the world population value (7,256,490,011). A quick search of world population trends reveals that the CIA could have compiled the facts data between 2014 and 2015.<br>
<img src="./images/world_population.png" 
     height=300px 
     width=300px 
     align='left'
     style="margin-left:auto; margin-right:auto"/>
<div style='clear:both;'></div>

_**Image source:** [Worldometer](https://www.worldometers.info/world-population/world-population-by-year/)_.

With these outliers in mind, we can compute  a more appropriate descriptive statistics.

In [8]:
%%sql 
SELECT MIN(population) AS min_pop,
       CAST(AVG(population) AS Integer) AS avg_pop,
       MAX(population) AS max_pop
  FROM facts
 WHERE population != 0 
   AND name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,avg_pop,max_pop
48,32377011,1367485388


The minimum population is 48, and the maximum is over 1.3 billion—the average population centers around 32 million inhabitants. We can now identify the highest and least populated countries or entities.

### What are the top 5 countries in terms of total population?
---

In [9]:
%%sql 
SELECT name AS top_10_populated,
       population
  FROM facts
 WHERE population <> 0 
   AND name <> 'World'
 ORDER BY 2 DESC
 LIMIT 5;

 * sqlite:///factbook.db
Done.


top_10_populated,population
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674


The European Union is not a country; it is better to filter out the EU. As an extra step, we will calculate the proportion of the world's population contained in each country.

In [10]:
%%sql 
SELECT name AS "10 Most Populated Countries",
       population,
       ROUND(
              CAST(population AS Float) / 
              CAST((SELECT MAX(population) FROM facts) AS Float)
             , 3) AS world_proportion
  FROM facts
 WHERE population <> 0 
   AND name NOT IN ('World', 'European Union')
 ORDER BY 2 DESC
 LIMIT 5;

 * sqlite:///factbook.db
Done.


10 Most Populated Countries,population,world_proportion
China,1367485388,0.188
India,1251695584,0.172
United States,321368864,0.044
Indonesia,255993674,0.035
Brazil,204259812,0.028


>- China is the world's most populated country, with about 1.37 billion people. India follows behind with about 1.26 billion people. **36%** of the world's total population comes from China (18.8%) and India (17.2%) alone.
>- The United States (4.4% of the world total), Indonesia (3.5%), and Brazil (2.8%) follow far behind and constitute a combined **11%** of the world's population. Altogether these top 5 countries contribute **46%** to the global population.